In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2757913881147328161, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 408851848666146759
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
import sys
import time
import math
import os
import pandas as pd
import numpy as np

import keras
from keras import layers, metrics, models, optimizers
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, concatenate, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [21]:
batch_size = 16
img_rows, img_cols = 256, 256
input_shape = (img_rows, img_cols, 3)
epochs = 50

In [6]:
train_dir = 'data_augmented/cvc300_train/'
validation_dir = 'data_augmented/cvc300_validation/'
test_dir = 'data_augmented/cvc300_test/'

image_folder = 'images'
masks_folder = 'masks'

In [14]:
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda


def Unet(img_size):
    inputs = Input((img_size, img_size, 3))
    s = Lambda(lambda x: x / 255)(inputs)

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

In [15]:
seed = 1142
data_gen_args = dict(rescale=1. / 255, featurewise_center=True, featurewise_std_normalization=True)
image_datagen = ImageDataGenerator(**data_gen_args)
masks_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    train_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    train_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_image_generator = image_datagen.flow_from_directory(
    validation_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_mask_generator = masks_datagen.flow_from_directory(
    validation_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_image_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_mask_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

Found 26112 images belonging to 1 classes.
Found 26112 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 28 images belonging to 1 classes.
Found 28 images belonging to 1 classes.


In [16]:
train_generator = zip(image_generator, mask_generator)
validation_generator = zip(validation_image_generator, validation_mask_generator)
test_generator = zip(test_image_generator, test_mask_generator)

In [17]:
nb_train_samples = len(image_generator.filenames)
nb_validation_samples = len(validation_image_generator.filenames)
nb_test_samples = len(test_image_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

nb_train_samples: 26112
nb_validation_samples: 8704
nb_test_samples: 28

predict_size_train: 1632
predict_size_validation: 544
predict_size_test: 2


In [18]:
smooth = 1.
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def Specificity(y_true, y_pred):
    true_negatives = K.abs(y_pred)- K.abs(y_true)
    return ((true_negatives+smooth)/(y_pred+ smooth))

def Sensitivity(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return ((y_pred+smooth)/ (y_true+smooth))

def Jaccard_index(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall))

In [19]:
model = Unet(img_size = 256)
model.summary()
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer= adam_opt, loss=dice_coef_loss, 
              metrics=[dice_coef, 'acc','mse', Jaccard_index, Specificity, Sensitivity, precision, recall, f1score])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 256, 256, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 256, 256, 16) 448         lambda_2[0][0]                   
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 256, 256, 16) 0           conv2d_20[0][0]                  
__________________________________________________________________________________________________
conv2d_21 

In [23]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    verbose = 2,
                    validation_data = validation_generator,
                    validation_steps = nb_validation_samples//batch_size)

Epoch 1/50
 - 365s - loss: -1.7285e-01 - dice_coef: 0.1728 - acc: 0.6534 - mean_squared_error: 0.2224 - Jaccard_index: 0.0952 - Specificity: 0.9655 - Sensitivity: 1.1952 - precision: 0.1168 - recall: 0.5148 - f1score: 0.1877 - val_loss: -1.4462e-01 - val_dice_coef: 0.1446 - val_acc: 0.2620 - val_mean_squared_error: 0.6505 - val_Jaccard_index: 0.0782 - val_Specificity: 0.9709 - val_Sensitivity: 1.6547 - val_precision: 0.0873 - val_recall: 0.9884 - val_f1score: 0.1599
Epoch 2/50
 - 363s - loss: -1.9235e-01 - dice_coef: 0.1923 - acc: 0.7210 - mean_squared_error: 0.1474 - Jaccard_index: 0.1072 - Specificity: 0.9627 - Sensitivity: 1.1120 - precision: 0.1506 - recall: 0.3924 - f1score: 0.2072 - val_loss: -1.7526e-01 - val_dice_coef: 0.1753 - val_acc: 0.4301 - val_mean_squared_error: 0.4634 - val_Jaccard_index: 0.0965 - val_Specificity: 0.9681 - val_Sensitivity: 1.4587 - val_precision: 0.1091 - val_recall: 0.8869 - val_f1score: 0.1935
Epoch 3/50
 - 364s - loss: -2.7394e-01 - dice_coef: 0.2739

Epoch 19/50
 - 362s - loss: -6.2430e-01 - dice_coef: 0.6243 - acc: 0.8224 - mean_squared_error: 0.0271 - Jaccard_index: 0.4569 - Specificity: 0.9674 - Sensitivity: 1.0030 - precision: 0.7445 - recall: 0.6168 - f1score: 0.6704 - val_loss: -3.5900e-01 - val_dice_coef: 0.3590 - val_acc: 0.7087 - val_mean_squared_error: 0.1514 - val_Jaccard_index: 0.2198 - val_Specificity: 0.9657 - val_Sensitivity: 1.1393 - val_precision: 0.2636 - val_recall: 0.8164 - val_f1score: 0.3963
Epoch 20/50
 - 361s - loss: -6.3511e-01 - dice_coef: 0.6351 - acc: 0.8282 - mean_squared_error: 0.0260 - Jaccard_index: 0.4683 - Specificity: 0.9677 - Sensitivity: 1.0025 - precision: 0.7576 - recall: 0.6249 - f1score: 0.6810 - val_loss: -3.6065e-01 - val_dice_coef: 0.3606 - val_acc: 0.7100 - val_mean_squared_error: 0.1514 - val_Jaccard_index: 0.2210 - val_Specificity: 0.9661 - val_Sensitivity: 1.1398 - val_precision: 0.2639 - val_recall: 0.8235 - val_f1score: 0.3975
Epoch 21/50
 - 364s - loss: -6.4495e-01 - dice_coef: 0.6

Epoch 37/50
 - 364s - loss: -7.6514e-01 - dice_coef: 0.7651 - acc: 0.8324 - mean_squared_error: 0.0153 - Jaccard_index: 0.6212 - Specificity: 0.9713 - Sensitivity: 1.0019 - precision: 0.8783 - recall: 0.7769 - f1score: 0.8232 - val_loss: -5.3298e-01 - val_dice_coef: 0.5330 - val_acc: 0.7743 - val_mean_squared_error: 0.0770 - val_Jaccard_index: 0.3659 - val_Specificity: 0.9673 - val_Sensitivity: 1.0690 - val_precision: 0.4408 - val_recall: 0.8747 - val_f1score: 0.5833
Epoch 38/50
 - 365s - loss: -7.7243e-01 - dice_coef: 0.7724 - acc: 0.8333 - mean_squared_error: 0.0149 - Jaccard_index: 0.6307 - Specificity: 0.9714 - Sensitivity: 1.0020 - precision: 0.8828 - recall: 0.7874 - f1score: 0.8312 - val_loss: -5.3928e-01 - val_dice_coef: 0.5393 - val_acc: 0.7757 - val_mean_squared_error: 0.0743 - val_Jaccard_index: 0.3716 - val_Specificity: 0.9675 - val_Sensitivity: 1.0661 - val_precision: 0.4487 - val_recall: 0.8728 - val_f1score: 0.5897
Epoch 39/50
 - 363s - loss: -7.7442e-01 - dice_coef: 0.7

In [24]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])

Validation Score:  -0.6461014592647553
Validation Accuracy:  0.6461014592647553


In [25]:
prediction = model.predict_generator(test_generator, verbose=1, steps=nb_test_samples)

28/28 [==============================] - 5s 167ms/step


In [26]:
prediction.shape

(392, 256, 256, 1)

In [27]:
test_results = model.evaluate_generator(test_generator, steps=50)

print(model.metrics_names)
print(test_results)

['loss', 'dice_coef', 'acc', 'mean_squared_error', 'Jaccard_index', 'Specificity', 'Sensitivity', 'precision', 'recall', 'f1score']
[-0.689056271144322, 0.6890562742097037, 0.9600789090565273, 0.0392681960016489, 0.5264078148773739, 0.9725590552602495, 1.0321390049798147, 0.5612502528939929, 0.8985275227682931, 0.6889424565860204]


In [28]:
coef = np.array(history.history['Jaccard_index'])
val_coef = np.array(history.history['val_Jaccard_index'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))

Training co-effiency    : 0.6893735292787645;
Validation co-effiency : 0.4931637432754916


In [ ]:
import skimage.io as io

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

os.mkdir('05.U-Net2_results_dir')

In [ ]:
saveResult('05.U-Net2_results_dir',prediction)

In [ ]:
def normalizeData(img,mask):
    mean = np.mean(img)
    std = np.std(img)
    img -= mean
    img /= std
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img,mask)

test_data_dir= "data_augmented/cvc300_test/images/"
test_data_mask_dir = "data_augmented/cvc300_test/masks/"
test_data_pred_dir = "05.U-Net2_results_dir"
img_rows, img_cols = 256, 256

In [ ]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

file_names = next(os.walk(test_data_dir))[2]
scores = []
for file in file_names:
    grey_img = load_img(os.path.join(test_data_dir,file), target_size=(img_rows, img_cols), grayscale=False)
    mask_img = load_img(os.path.join(test_data_mask_dir,file), target_size=(img_rows, img_cols), grayscale=True)
    img = img_to_array(grey_img)
    img_mask = img_to_array(mask_img)
    
    img, img_mask = normalizeData(img, img_mask)
    img = np.reshape(img,(1,)+img.shape)
    
    pred = model.predict([img])
    sess = tf.Session()
    score = sess.run(Jaccard_index(img_mask, pred))
    print("{} -- jaccard index: {}".format(file,score))
    scores.append([file,score])

    result_img = array_to_img(pred[0] * 255 )
    result_img.save(os.path.join(test_data_pred_dir, file.split('.')[0] + '_predict.jpg'))

with open("unet_test_result.csv", 'w') as f:
    f.write("filename, jaccard_index\n")
    for i in range(len(scores)):
        #print(scores[i])
        f.write("{},{}\n".format(scores[i][0], scores[i][1]))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/102_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray")

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray", vmin = 0.0, vmax= 0.09)

In [ ]:
results_ =  np.squeeze(np.multiply(prediction[1], 255))
print(results_)

plt.imshow(results_, cmap = "gray", vmin = 0.0, vmax= 0.7)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/35_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice_coeff')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()